In [1]:
# KaggleX Cohert 4 Skill Assessment Challenge

#### import libraries

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split

2024-06-14 16:21:19.691199: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 16:21:19.691310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 16:21:19.963072: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#### Dataset Paths

In [3]:
# Dataset paths
train_path = "/kaggle/input/kagglex-car-price-prediction-dataset/train.csv"
test_path = "/kaggle/input/kagglex-car-price-prediction-dataset/test.csv"

In [4]:
train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

In [5]:
train_set.shape

(54273, 13)

In [6]:
test_set.shape

(36183, 12)

In [7]:
test_set.shape[0] * 0.3

10854.9

In [8]:
train_set.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [9]:
test_set.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
2,54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
3,54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
4,54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


In [10]:
train_set.any().isna()

id              False
brand           False
model           False
model_year      False
milage          False
fuel_type       False
engine          False
transmission    False
ext_col         False
int_col         False
accident        False
clean_title     False
price           False
dtype: bool

In [11]:
test_set.any().isna()

id              False
brand           False
model           False
model_year      False
milage          False
fuel_type       False
engine          False
transmission    False
ext_col         False
int_col         False
accident        False
clean_title     False
dtype: bool

## Shuffle Train Set

In [12]:
# train_set = train_set.sample(frac=1)
# train_set.head()

## Separate Features and Labels

In [13]:
# Set features similar to the test set only without id column
features = train_set.copy()
features.pop("id")

# Remove price column in features and save it in labels
labels = features.pop("price")
features.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes
1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes
2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes
3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes
4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes


In [14]:
labels.head()

0    11000
1     8250
2    15000
3    63500
4     7850
Name: price, dtype: int64

## Split Dataset

In [15]:
# Split the dataset into train (70%) and remain (30%) subsets
train_features, remain_features, train_labels, remain_labels = sklearn.model_selection.train_test_split(
    features, 
    labels,
    test_size=0.3,
    shuffle=True
)

# Split the remain subset into valid (70%) and test (30%) subset
valid_features, test_features, valid_labels, test_labels = sklearn.model_selection.train_test_split(
    remain_features, 
    remain_labels,
    test_size=0.3,
    shuffle=True
)

In [16]:
# Check if indices of features and labels still match
print(train_features.head().index)
print(train_labels.head().index)

print(test_features.head().index)
print(test_labels.head().index)

print(valid_features.head().index)
print(valid_labels.head().index)

Index([19718, 19542, 15097, 28385, 10939], dtype='int64')
Index([19718, 19542, 15097, 28385, 10939], dtype='int64')
Index([22171, 15793, 53614, 21034, 4211], dtype='int64')
Index([22171, 15793, 53614, 21034, 4211], dtype='int64')
Index([13921, 25653, 35840, 21320, 51096], dtype='int64')
Index([13921, 25653, 35840, 21320, 51096], dtype='int64')


In [17]:
# Check if dataset were split correctly
dataset_size = len(train_set)
print(dataset_size * 0.7, len(train_features), len(train_labels))
print(dataset_size * 0.3 * 0.7, len(valid_features), len(valid_labels))
print(dataset_size * 0.3 * 0.3, len(test_features), len(test_labels))

# Check if total of the subsets the same as dataset_size
dataset_size, len(train_features) + len(valid_features) + len(test_features), len(train_labels) + len(valid_labels) + len(test_labels)

37991.1 37991 37991
11397.33 11397 11397
4884.57 4885 4885


(54273, 54273, 54273)

TODO
- build model
- separate engine into different parameters
- remove id on train
- train model
- group lines

In [18]:
# train_subset.shape

In [19]:
# valid_subset.shape

In [20]:
# test_subset.shape